In [1]:
import pandas as pd
import numpy as np
import facebook as fb
import telebot
from datetime import datetime
from datetime import date
import re

In [8]:
access_token = 'EAACEdEose0cBAAzKg9L92ycTLmO9jF0OOh44DKNKgTICSE75NKDCWz4MOEXgSYFimrBWj9GYxRtZBEnRRk0KfXgYtj7HZBfXTipoVZAnByLyZB2QL5OBpn9dmZCpUj1cEMFgQUvceZAypnTCQZAeZBn40xZAczWkU5X1TwidZAPXV8E3odwVPKKZCasfKElHJeWX6YZD'
pageID = '290525360959001'

In [9]:
def fb_parsing(access_token):
    graph = fb.GraphAPI(access_token)
    profile = graph.get_object(pageID)
    posts = graph.get_connections(id=profile['id'], connection_name='feed')['data']
    months = {'января', 'февраля', 'марта', 'апреля', 'мая', 'июня', 'июля', 'августа', 'сентября', 'октября', 'ноября',
              'декабря'}
    monthsOrder = {'января': 1, 'февраля': 2, 'марта': 3, 'апреля': 4, 'мая': 5, 'июня': 6,  #
                   'июля': 7, 'августа': 8, 'сентября': 9, 'октября': 10, 'ноября': 11, 'декабря': 12}
    eventsEndWordsSet = {'до', 'по'}
    futureEvents_new = []
    try:
        futureEvents = list(pd.read_csv('futureEvents_' + pageID + '.csv', sep=',').to_dict(orient = 'index').values())
    except:
        futureEvents = []
        file = open('futureEvents_' + pageID + '.csv', 'w')
        file.close()
    for post in posts:
        getStartDate = 0
        getEndDate = 0
        try:
            text = post['message']
            text = text.replace('\n', ' ')
            wordsList = text.split()
            for wordPos in range(len(wordsList)):
                wordsList[wordPos] = wordsList[wordPos].strip('.').strip(',').strip('-')
            #print(wordsList)
            for wordPos in range(len(wordsList)):
                if wordsList[wordPos] in months and re.match(r'[0-9][0-9]?\b', wordsList[wordPos - 1]):
                    if len(wordsList) > wordPos + 2 and \
                            len({wordsList[wordPos + 1], wordsList[wordPos + 2], wordsList[(wordPos + 2) % len(wordsList)]} \
                            & {'года', 'году'}) != 0:
                                pass
                                #print('!', wordsList[wordPos - 1], wordsList[wordPos], wordsList[wordPos + 1])
                    else:
                        #print(wordsList[wordPos - 1], wordsList[wordPos])
                        eventMonth = monthsOrder[wordsList[wordPos]]
                        eventYear = datetime.now().year
                        if (eventMonth < datetime.now().month) and (eventMonth < 3):
                            eventYear = eventYear + 1
                        eventDay = int(wordsList[wordPos - 1])

                        if getStartDate == 0:
                            getStartDate = 1
                            eventStart = date(eventYear, eventMonth, eventDay)
                        try:
                            if (wordsList[wordPos - 2] in eventsEndWordsSet) and getEndDate == 0:
                                getEndDate = 1
                                eventEnd = date(eventYear, eventMonth, eventDay)
                        except:
                            pass
                        # print(datetime.date(datetime.now()), eventStart)
            if (getStartDate == 0):
                pass
            else:
                if (getEndDate == 0):
                    eventEnd = eventStart
                if (datetime.date(datetime.now()) <= eventStart) and (getStartDate != 0):
                #if getStartDate != 0:
                    futureEvents_new.append({'id': post['id'], 'from': eventStart, 'to': eventEnd, 'main': text})
                    # print('!', futureEvents_new)
        except KeyError:
            pass

    ID_set = set()
    for e in futureEvents:
        ID_set = ID_set | {e['id']}
    for newPost in futureEvents_new:
        if newPost['id'] not in ID_set:
            futureEvents.append(newPost)
            ID_set = ID_set | {newPost['id']}

    pd.DataFrame(futureEvents).to_csv('futureEvents_' + pageID + '.csv', index=False, sep= ',')

In [10]:
def get_relevant_events(date_str):
    # print(date_str)
    date_str = date_str.replace('/date ', '')
    nums = date_str.split('.')
    date_date = date(int(nums[2]), int(nums[1]), int(nums[0]))
    relevantEvents = []
    futureEvents = list(pd.read_csv('futureEvents_' + pageID + '.csv', sep=',').to_dict(orient = 'index').values())
    # print(futureEvents)
    for event in futureEvents:
        from_date = datetime.strptime(event['from'], '%Y-%m-%d').date()
        to_date = datetime.strptime(event['to'], '%Y-%m-%d').date()
        # print(from_date, to_date)
        if date_date >= from_date and date_date <= to_date:
            relevantEvents.append(event['main'])
    return relevantEvents

In [11]:
telegramToken = '413446284:AAGHqs63JjdCOCjTngZvohHtYp5ytBuPLkQ'
bot = telebot.TeleBot(telegramToken)

In [12]:
@bot.message_handler(commands=['start'])
def handle_start(message):
   bot.send_message(message.chat.id, 'Привет! Я подскажу, куда сходить в Москве. Напиши /date и дату в формате 17.5.2017.')

@bot.message_handler(commands=['date'], content_types=['text'])
def search_events(message):
    eventsToShow = get_relevant_events(message.text)
    if len(eventsToShow) == 0:
        bot.send_message(message.chat.id, 'There are no events for you, looser.')
    else:
        for text in eventsToShow:
            bot.send_message(message.chat.id, text)

@bot.message_handler(func=lambda m: True)
def all_others(message):
	bot.reply_to(message, 'Please use the command /date')

In [ ]:
if __name__ == '__main__':
    fb_parsing(access_token)
    bot.polling(none_stop=True)